In [119]:
class Vektor:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        x, y = self.koordinate()
        return ('{}(x = {}, y = {})'.format(self.__class__.__name__, x, y))
    
    def __add__(self, other):
        x1 = self.x
        x2 = other.x
        y1 = self.y
        y2 = other.y
        return(Vektor(x1 + x2, y1 + y2))
    
    def __sub__(self, other):
        x1 = self.x
        x2 = other.x
        y1 = self.y
        y2 = other.y
        return(Vektor(x1 - x2, y1 - y2))
    
    def __rmul__(self, konstanta):
        x1 = konstanta * self.x
        y1 = konstanta * self.y
        return(Vektor(x1, y1))
    
    def koordinate(self):
        return self.x, self.y
    
class Premica:
    def __init__(self, u1, u2): # u tocka; rabili bomo koordinate
        u1x, u1y = u1.koordinate()
        u2x, u2y = u2.koordinate()
        if u2x - u1x != 0:
            self.koef = (u2y -u1y) / (u2x - u1x)
            self.n = u1y - self.koef * u1x
        else:
            self.koef = "NaN"
            self.x = u1x
        
    def parametri(self):
        if self.koef != "NaN":
            return self.koef, self.n
        else:
            return self.x
    
    def __repr__(self):
        if self.koef != "NaN":
            k, n = self.parametri()
            return ('{}(y = {} * x + {})'.format(self.__class__.__name__, k, n))
        else:
            x = self.parametri()
            return ('{}(x = {})'.format(self.__class__.__name__, x))
    
class U:
    def __init__(self, a, b):
        self.a = a # a konstanta ... koliko "prvih" vektorjev uporabimo
        self.b = b
        
    def __repr__(self):
        a,b = self.podatki()
        return ('{}({}a, {}b)'.format(self.__class__.__name__, a, b))
    
    def podatki(self):
        return self.a, self.b
    
    def koordinate(self):
        vektor = self.a * A1 + self.b * A2
        koordinate = vektor.koordinate()
        return [koordinate[0], koordinate[1]]
    
    def sosedi(self):
        sosedi = set()
        sosedi.add(V(self.a, self.b))
        sosedi.add(V(self.a + 1, self.b - 1))
        sosedi.add(V(self.a, self.b - 1))
        return sosedi
    
    def na(self, premica): 
        x, y = self.koordinate()
        if premica.koef != "NaN":
            if y == premica.koef * x + premica.n:
                return True
            else:
                return False
        # navpicnica:
        else:
            if x == premica.x:
                return True
            else:
                return False

class V:
    def __init__(self, a, b):
        self.a = a # a konstanta ... koliko "prvih" vektorjev uporabimo
        self.b = b
        
    def __repr__(self):
        a,b = self.podatki()
        return ('{}({}a, {}b)'.format(self.__class__.__name__, a, b))
    
    def podatki(self):
        return self.a, self.b
    
    def koordinate(self):
        vektor = self.a * A1 + self.b * A2 + Vektor(1, 0)
        koordinate = vektor.koordinate()
        return [koordinate[0], koordinate[1]]
    
    def sosedi(self):
        sosedi = set()
        sosedi.add(U(self.a, self.b))
        sosedi.add(U(self.a - 1, self.b + 1))
        sosedi.add(U(self.a, self.b + 1))
        return sosedi
    
    def na(self, premica): 
        x, y = self.koordinate()
        if premica.koef != "NaN":
            if y == premica.koef * x + premica.n:
                return True
            else:
                return False
        # navpicnica:
        else:
            if x == premica.x:
                return True
            else:
                return False
            
        

A1 = Vektor(0, - sqrt(3))
A2 = Vektor(3 / 2, - sqrt(3) / 2)

def grid(k, l, m, n):
    # oglisca = ([0, 0], [k, l], [m, n], [k + m, l + n])
    # print U(0,0).koordinate(), U(k, l).koordinate(), U(m, n).koordinate(), U(k + m, l + n).koordinate()
    # lahko bi dali vsa vozlisca kar v mnozico??
    mina = min(0, k, m, k + m)
    maxa = max(0, k, m, k + m)
    minb = min(0, l, n, l + n)
    maxb = max(0, l, n, l + n)
    
    matrika = []
    for i in range(mina, maxa + 1):
        vrstica = []
        for j in range(minb, maxb + 1):
            vrstica += [U(i, j)]
            # print U(i,j), U(i, j).koordinate()
        matrika += [vrstica]
        
    return matrika

from matplotlib.path import Path
# mogoce to celo dela?? # trenutno vrne vse tocke v paralelogramu. odstraniiti je potrebno se tiste na obeh premicah
# premisliti je treba se tiste pogoje, ce je use uredu... to, da lezi na premici pa to
# (sem zdej naredu, lih prej k sm su u lj, pa nism preverju)

def paralelogram(k, l, m, n):
    G = grid(k, l, m, n)
    oglisca = [U(0,0).koordinate(), U(k, l).koordinate(), U(k + m, l + n).koordinate(), U(m, n).koordinate()]
    p = Path(oglisca[::-1])
    premica1 = Premica(U(0, 0), U(k, l))
    premica2 = Premica(U(0, 0), U(m, n))
    Uji = set()
    Vji = set()
    for i in range(len(G)):
        for j in range(len(G[0])):
            izbrana = G[i][j]
            tockaU = [izbrana.koordinate()]
            a = izbrana.a
            b = izbrana.b
            izbranaV = V(a, b)
            tockaV = [izbranaV.koordinate()]
            vsebuje_u = p.contains_points(tockaU)[0] # ker vrne array[True]
            vsebuje_v = p.contains_points(tockaV)[0] #####ZAKAJ JE TREBA TUKEJ RADIUS?
            if vsebuje_u and not izbrana.na(premica1) and not izbrana.na(premica2): 
                Uji.add(G[i][j])
            elif vsebuje_v and not izbranaV.na(premica1) and not izbranaV.na(premica2):
                Vji.add(izbranaV)
    return Uji, Vji

def vsebovana(tocka, M):
    for m in M:
        if m.__class__.__name__ == tocka.__class__.__name__ and m.a == tocka.a and m.b == tocka.b:
            return True
    return False

def povezave(tocka, k, l, m, n):
    Uij, Vij = paralelogram(k, l, m, n)
    povezave = set()
    for s in tocka.sosedi():
        if vsebovana(s, union(Vij, Uij)):
            povezave.add(s)
        elif s.__class__.__name__ == 'V' and vsebovana(V(s.a + k, s.b + l), Uij):
            s1 = U(s.a + k, s.b + l)
            povezave.add(s1)
        elif s.__class__.__name__ == 'V' and vsebovana(V(s.a + m, s.b + m), Vij):
            s1 = U(s.a + m, s.b + n)
            povezave.add(s1)
        elif s.__class__.__name__ == 'U' and vsebovana(U(s.a + k, s.b + l), Vij):
            s1 = V(s.a + k, s.b + l)
            povezave.add(s1)
        elif s.__class__.__name__ == 'U' and vsebovana(U(s.a + m, s.b + n), Vij):
            s1 = V(s.a + m, s.b + n)
            povezave.add(s1)
    return povezave

#def graf(k,l,m,n):
#    Uij, Vij = paralelogram(7,-3,4,1)
#    Graph(union(Uij, Vij), )    

In [120]:
paralelogram(2,0, 0, 2)

({U(2a, 1b), U(1a, 1b)}, {V(2a, 0b), V(1a, 0b)})

In [115]:
tocka = U(1, 1)

In [116]:
tocka.sosedi()

{V(1a, 1b), V(2a, 0b), V(1a, 0b)}

In [117]:
povezave(tocka, 2,0,0,2)

{V(2a, 0b), V(1a, 0b)}